## Introduction to our first task: 'Dogs vs Cats'
Same functionality as "tmp_lesson1-quick.ipynb" but need much more code, and you need to set lots of parameters very carefully

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
PATH = "data/dogscats/"
sz=224
batch_size=64

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [4]:
#CUDA_VISIBLE_DEVICES=0

In [5]:
train_data_dir = f'{PATH}train'
validation_data_dir = f'{PATH}valid'

In [6]:
# A good way to use them is to copy from the Keras documentation, and we are doing so here.
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 2 outcomes here of cat or dog, we use "binary"; multiple outcomes will use "categorical"
# default the shuffle=True
train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

# shuffle=False, or the data and lables will not be matched
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    shuffle=False,
    target_size=(sz, sz),
    batch_size=batch_size, class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False)
# Unlike Fast.ai, based on the above line of base model, you need to contruct on top of that manually as below lines for the
# layers you want to add
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=4,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/3
359/359 [==============================] - 3650s 10s/step - loss: 0.3527 - acc: 0.9368 - val_loss: 0.2585 - val_acc: 0.9647
Epoch 2/3
359/359 [==============================] - 3652s 10s/step - loss: 0.4253 - acc: 0.9547 - val_loss: 0.2441 - val_acc: 0.9788
Epoch 3/3
359/359 [==============================] - 3651s 10s/step - loss: 0.4671 - acc: 0.9619 - val_loss: 0.8953 - val_acc: 0.9396
Wall time: 3h 2min 33s


In [12]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])    

In [13]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=1, workers=3,
        validation_data=validation_generator, validation_steps=validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 5053s 14s/step - loss: 0.4227 - acc: 0.9581 - val_loss: 0.4423 - val_acc: 0.9632
Wall time: 1h 24min 15s
